## Лабораторна робота №2
Виконав студент групи ФІ-31 Дремко Олександр

In [49]:
import pandas as pd
import os
from urllib import request
from datetime import datetime
from shutil import rmtree

1. Для кожної із адміністративних одиниць України завантажити тестові
структуровані файли, що містять значення VHI-індексу. Ця процедура має бути
автоматизована, параметром процедури має бути індекс (номер) області. При зберіганні файлу до його імені потрібно додати дату та час завантаження. Передбачити повторні запуски скрипту, довантаження нових даних та колізію даних


In [50]:
# Перевірка папки на порожність та очищення
def check_folder(folder_path):
    # Якщо папки не існує то створюємо її
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        
    # Якщо папка існує і не порожня то очищаємо її
    elif os.listdir(folder_path):
        rmtree(folder_path)
        os.makedirs(folder_path)

# Завантаження даних для області за id
def download_data(province_id):
    url = f'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={province_id}&year1=1981&year2=2024&type=Mean'
    try:
        vhi = request.urlopen(url).read().decode('utf-8')
        now = datetime.now()
        formatted_time = now.strftime('%d%m%Y%H%M%S')
        file_name = f'data/vhi_id_{province_id}_{formatted_time}.csv'
        with open(file_name, 'w') as f:
            f.write(vhi)
        print(f'Data for province {province_id} saved as {file_name}')
    except Exception as e:
        print(f'Error loading file for province {province_id}: {e}')

folder_path = 'data'
#check_folder(folder_path)

#for province_id in range(1, 28):
#    download_data(province_id)


2. Зчитати завантажені текстові файли у фрейм. Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли

In [51]:
def create_df(folder_path):
    all_dfs = []  # Список для збереження всіх DataFrame

    # Обхід усіх файлів у папці
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            
            # Витягуємо area з імені файлу, розділяючи підкресленням (_)
            try:
                area = int(filename.split('_')[2])
            except (IndexError, ValueError):
                print(f"Не вдалося витягти area з файлу: {filename}")
                continue

            # Завантаження та очищення даних
            try:
                # Пропускаємо метадані та зчитуємо дані починаючи з заголовка
                df = pd.read_csv(file_path, skiprows=2, skipinitialspace=True)
                
                # Видаляємо порожні стовпці, якщо вони є
                df = df.dropna(axis=1, how="all")
                
                # Виправляємо назви стовпців
                df.columns = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI']
                
                # Додаємо ідентифікатор області
                df['area'] = area
                
                # Перетворюємо дані на числові типи
                for col in ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI']:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
                
                # Видаляємо рядки з NaN у стовпці Year
                df = df[df['Year'].notna()]

                # Видаляємо рядки з -1
                df = df[df['VHI'] != -1] 
                
                # Додаємо очищений DataFrame до списку
                all_dfs.append(df)
                
            except pd.errors.EmptyDataError:
                print(f"Файл порожній або не вдалося прочитати: {filename}")

    # Об’єднання всіх DataFrame в один
    if all_dfs:
        combined_df = pd.concat(all_dfs, ignore_index=True)
    else:
        print("Не вдалося об'єднати DataFrame: список порожній")
        return pd.DataFrame()  # Повертаємо порожній DataFrame у разі помилки

    # Виводимо інформацію про об'єднаний DataFrame
    print("Об'єднаний DataFrame створено")
    return combined_df

In [52]:
df = create_df(folder_path)
#print(df.tail())
#print(df.head())

filtered_df = df[df['area'] == 1]
print(filtered_df.head())

Об'єднаний DataFrame створено
         Year  Week    SMN     SMT    VCI    TCI    VHI  area
21750  1982.0   2.0  0.054  262.29  46.83  31.75  39.29     1
21751  1982.0   3.0  0.055  263.82  48.13  27.24  37.68     1
21752  1982.0   4.0  0.053  265.33  46.09  23.91  35.00     1
21753  1982.0   5.0  0.050  265.66  41.46  26.65  34.06     1
21754  1982.0   6.0  0.048  266.55  36.56  29.46  33.01     1


3. Реалізувати окрему процедуру, яка змінить індекси областей, які використані на порталі NOAA з англійської абетки, на українську (старі індекси на нові)

In [53]:
def replace_province_ids(df):
    # Перевірка, чи є колонка area в датафреймі
    if 'area' not in df.columns:
        print("Помилка: колонка 'area' відсутня.")
        return df
    
    # Словник для заміни ідентифікаторів областей
    new_ids = {
        1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20,
        10: 21, 11: 9, 12: 9, 13: 10, 14: 11, 15: 12, 16: 13, 17: 14,
        18: 15, 19: 16, 20: 26, 21: 17, 22: 18, 23: 6, 24: 1, 25: 2,
        26: 7, 27: 5
    }
    # Замінюємо значення у колонці area відповідно до словника
    dfc = df.copy()
    dfc["area"] = dfc["area"].replace(new_ids)
    return dfc

In [54]:
uk_df = replace_province_ids(df)
print(uk_df.head())
print(df.head())

     Year  Week    SMN     SMT    VCI    TCI    VHI  area
0  1982.0   2.0  0.063  261.53  55.89  38.20  47.04    21
1  1982.0   3.0  0.063  263.45  57.30  32.69  44.99    21
2  1982.0   4.0  0.061  265.10  53.96  28.62  41.29    21
3  1982.0   5.0  0.058  266.42  46.87  28.57  37.72    21
4  1982.0   6.0  0.056  267.47  39.55  30.27  34.91    21
     Year  Week    SMN     SMT    VCI    TCI    VHI  area
0  1982.0   2.0  0.063  261.53  55.89  38.20  47.04    10
1  1982.0   3.0  0.063  263.45  57.30  32.69  44.99    10
2  1982.0   4.0  0.061  265.10  53.96  28.62  41.29    10
3  1982.0   5.0  0.058  266.42  46.87  28.57  37.72    10
4  1982.0   6.0  0.056  267.47  39.55  30.27  34.91    10


#### 4. Реалізувати процедури для формування вибірок наступного виду:

1. Ряд VHI для області за вказаний рік

In [55]:
def vhi_series(df, province_id, year):
    # Перевірка, чи містить датафрейм вказані колонки
    if 'area' not in df.columns or 'Year' not in df.columns or 'VHI' not in df.columns:
        print("Помилка: відсутні потрібні колонки.")
        return pd.Series(dtype='float64')

    # Фільтруємо дані для заданої області та року
    vhi_data = df[(df['area'] == province_id) & (df['Year'] == year)]['VHI']
    
    # Перевірка, чи є дані для вказаних параметрів
    if vhi_data.empty:
        print(f"Дані для області {province_id} у {year} році відсутні.")
        return pd.Series(dtype='float64')
    
    return vhi_data

In [56]:
vhi_series = vhi_series(df, 24, 2005)
print(vhi_series.head())

33771    56.65
33772    55.96
33773    56.88
33774    58.05
33775    58.71
Name: VHI, dtype: float64


2. Пошук екстремумів (min та max) для вказаних областей та років,
середнього, медіани

In [57]:
def get_min_max_mean_median(df, areas, years):
    # Перевірка наявності колонок
    if 'area' not in df.columns or 'Year' not in df.columns or 'VHI' not in df.columns:
        print("Помилка: відсутні потрібні колонки.")
        return None, None, None, None
    
    # Фільтруємо дані для вказаних областей та років
    filtered_data = df[(df['area'].isin(areas)) & (df['Year'].isin(years))]['VHI']
    
    # Перевірка наявності даних
    if filtered_data.empty:
        print(f"Дані для областей {areas} за роки {years} відсутні.")
        return None, None, None, None

    # Розрахунок показників
    vhi_min = filtered_data.min()
    vhi_max = filtered_data.max()
    vhi_mean = filtered_data.mean()
    vhi_median = filtered_data.median()

    return vhi_min, vhi_max, vhi_mean, vhi_median

In [58]:
min_val, max_val, mean_val, median_val = get_min_max_mean_median(df, [1, 2], [2000, 2024])
print(f"Min: {min_val}, Max: {max_val}, Mean: {mean_val}, Median: {median_val}")

Min: 10.68, Max: 72.05, Mean: 44.6627659574468, Median: 44.96


3. Ряд VHI за вказаний діапазон років для вказаних областей

In [59]:
def vhi_series_range(df, areas, start_year, end_year):
    # Перевірка наявності потрібних колонок
    if 'area' not in df.columns or 'Year' not in df.columns or 'VHI' not in df.columns:
        print("Помилка: відсутні потрібні колонки.")
        return pd.Series(dtype='float64')
    
    # Фільтруємо за областями та роками
    vhi_data = df[(df['area'].isin(areas)) & (df['Year'].between(start_year, end_year))]['VHI']
    
    # Перевірка наявності даних
    if vhi_data.empty:
        print(f"Дані для областей {areas} за роки {start_year}-{end_year} відсутні.")
        return pd.Series(dtype='float64')

    return vhi_data

In [60]:
vhi_series_range = vhi_series_range(df, [24, 17], 2000, 2005)
print(vhi_series_range.head())

16130    34.57
16131    38.38
16132    38.26
16133    38.16
16134    38.20
Name: VHI, dtype: float64


4. Для всього набору даних виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні. Повернути роки, назви областей зекстремальними посухами та значення VHI

    VHI < 15 – посуха, інтенсивність якої від середньої до надзвичайної

In [79]:
def find_extreme_drought_years(df, percent):
    # Рахуємо кількість областей
    total_areas = df['area'].nunique()
    threshold_areas = total_areas * (percent / 100)
    
    # Фільтруємо дані для екстремальної посухи
    drought_df = df[df['VHI'] < 15]

    # Групуємо за роком і рахуємо кількість уражених областей кожного року
    drought_counts = drought_df.groupby('Year')['area'].nunique()

    # Вибираємо лише роки, де кількість уражених областей перевищує threshold_areas
    extreme_drought_years = drought_counts[drought_counts > threshold_areas].index

    # Формуємо результат
    result = drought_df[drought_df['Year'].isin(extreme_drought_years)][['Year', 'area']].drop_duplicates()
    
    # Перевіряємо, чи є результати
    if result.empty:
        print("Не знайдено років з екстремальними посухами, що перевищують заданий відсоток.")
        return pd.DataFrame()

    return result

In [80]:
extreme_droughts = find_extreme_drought_years(df, 10)  # Шукаємо роки, коли 20% областей зазнали посухи
print(extreme_droughts)


         Year  area
3123   2000.0    11
5296   2000.0    12
14320  2007.0    16
16498  2007.0    17
22698  2000.0     1
24874  2000.0    20
33574  2000.0    24
38250  2007.0    26
46953  2007.0     4
55325  2000.0     8
57821  2007.0     9
